# Logistic Regression

In [1]:
import sys
sys.path.append('/home/belinda/Documents/career/FHNW/MlLab2_V2/cas_ml_lab_cifar10/src/util')

In [2]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from experiments.util import load_data

from sklearn.model_selection import train_test_split

# Prepare data

In [3]:
# Load the data and split into features and labels
X_data, y_data = load_data()

In [4]:
# Split features and labels into train (X_train, y_train) and validation set (X_val, y_val).
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, random_state=42)

# Logistic Regression

Beginnen wir mit einer einfachen Logistischen Regression, die wir auf den gesamten Datensatz anwenden.

In [5]:
%%time

lr = LogisticRegression()
lr.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = lr.predict(X_train.reshape(-1, 32 * 32 * 3))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = lr.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train:  0.4294133333333333
Val:    0.4044
CPU times: user 3min 2s, sys: 19.4 s, total: 3min 22s
Wall time: 26.8 s


Zufälliges raten hätte eine erwartete Genauigkeit von 10%.
Unser erstes Modell hat eine Genauigkeit von 40% auf dem Validation Set und damit schon einmal deutlich besser als zufälliges raten.

Wir erhalten eine Warnung: `STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.`.
Diese Warnung bedeutet, dass die Logistic Regression noch nicht zum globalen Minimum konvergiert ist.
Sprich, wir könnten die Logistic Regression weiter trainieren und sie wird sich auf dem `Trainings Set` noch verbessern.
Dies kann man über den Parameter `max_iter=100` einstellen.

Im folgenden Code setzen wir die `max_iter=1000`:

In [6]:
%%time

lr_1k = LogisticRegression(max_iter=1000)
lr_1k.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = lr_1k.predict(X_train.reshape(-1, 32 * 32 * 3))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = lr_1k.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train:  0.49952
Val:    0.37056
CPU times: user 33min 31s, sys: 2min 56s, total: 36min 28s
Wall time: 5min 45s


Wir sehen, dass wir uns auf dem `Training Set` verbessern, aber anfangen uns auf dem `Validation Set` zu verschlechtern, ein Hinweis auf `Overfitting`.
Zudem sind wir immer noch nicht konvergiert, sprich müssten `max_iter` weiter erhöhen.

Nächste mögliche Schritte wären die `max_iter` zu erhöhen bis wir konvergieren und gleichzeitig die `Regularisierung` zu erhöhen (`C` verkleinern) gegen das `Overfitting`.

Oder die Art vom Modell gänzlich zu verändern und die jetzige `Logistische Regression` als `Baseline` zu verwenden.
Beispielsweise kann man das Problem mit dem Konvertieren auch mit einer Unsupervised Dimensionsreduktion wie `PCA` angehen.

# (Extra) Logistic Regression with saga

Man kann bei Logistic Regression den Lern-Algorithmus mittels dem `solver` Parameter verändern.
Default ist dieser auf `lbfgs` eingestellt, ein Verfahren das über die zweite Ableitung lernt (im Theorie Teil nicht angeschaut).
Man kann den `solver` auch auf `saga` einstellen, was mehr oder weniger dem Stochastic Gradient Descent entspricht vom Theorie Teil.
`saga` ist grundsätzlich für grosse Datensätze (ab ~100'000 Zeilen) schneller im konvergieren.
Hier ist `saga` aber nicht wirklich schneller.
Zudem kann man im sklearn nur wenig für die Stochastic Gradient Descent einstellen: Es fehlen Parameter für die Batch Size und Momentum.

In [7]:
%%time

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

lr_saga = Pipeline([
    ('std', StandardScaler()), # Für saga müssen wir die Inputs zuerst standartisieren
    ('clf', LogisticRegression(solver='saga'))
])

lr_saga.fit(X_train.reshape(-1, 32 * 32 * 3), y_train)

y_train_hat = lr.predict(X_train.reshape(-1, 32 * 32 * 3))
print("Train: ", accuracy_score(y_train_hat, y_train))

y_val_hat = lr.predict(X_val.reshape(-1, 32 * 32 * 3))
print("Val:   ", accuracy_score(y_val_hat, y_val))

/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Train:  0.4294133333333333
Val:    0.4044
CPU times: user 8min 15s, sys: 4.69 s, total: 8min 20s
Wall time: 8min 24s


# Predict prices for test set

TODO